In [ ]:
from visionlab.datasets import StreamingDataset
from litdata.streaming.cache import Dir

import io
import torch
import numpy as np
import pathlib
from PIL import Image
import torchvision
import cv2

In [ ]:
try:
    # https://github.com/lilohuang/PyTurboJPEG/blob/master/turbojpeg.py
    from turbojpeg import TurboJPEG, TJPF_RGB, TJPF_BGR
except:
    # https://github.com/loopbio/PyTurboJPEG/tree/loopbio
    from turbojpeg import TurboJPEG, TJPF
    TJPF_RGB = TJPF.RGB
    TJPF_BGR = TJPF.BGR
    
turbo = TurboJPEG()
turbo

In [ ]:
ds = StreamingDataset("s3://visionlab-datasets/imagenet100/pre-processed/s256-l512-jpgbytes-q100-streaming/val",
                      decode_images=False)
ds

In [ ]:
sample = ds[0]
image_bytes = sample['image']

In [ ]:
# convert numpy arrays to bytes
if isinstance(image_bytes, np.ndarray):
    print("tobytes")
    image_bytes = image_bytes.tobytes()

In [ ]:
pixel_format=TJPF_RGB
rgb_image_turbo = turbo.decode(image_bytes, pixel_format=pixel_format)
rgb_image_turbo.shape

In [ ]:
Image.fromarray(rgb_image_turbo)

In [ ]:
rgb_image_pil = Image.open(io.BytesIO(image_bytes))
rgb_image_pil = rgb_image_pil.convert('RGB')
rgb_image_pil.verify()
rgb_image_pil

In [ ]:
# Decode to BGR
bgr_image = cv2.imdecode(np.frombuffer(image_bytes, dtype=np.uint8), cv2.IMREAD_COLOR)
# Convert BGR to RGB
rgb_image_cv2 = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
Image.fromarray(rgb_image_cv2)

In [ ]:
from torchvision.io import decode_image, ImageReadMode

img_buffer = torch.frombuffer(image_bytes, dtype=torch.uint8)
rgb_image_torch = decode_image(img_buffer, mode=ImageReadMode.RGB)
rgb_image_torch.shape

In [ ]:
Image.fromarray(rgb_image_torch.permute(1, 2, 0).numpy())

In [ ]:
from torchvision.io import decode_jpeg

device = 'cuda' if torch.cuda.is_available() else 'cpu'
img_cuda = decode_jpeg(img_buffer, device=device)